In [2]:
import tensorflow as tf
import numpy as np
from solver import CaptioningSolver
from model import CaptionGenerator
from cs231n.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from cs231n.image_utils import image_from_url
%load_ext autoreload
%autoreload 2

In [3]:
data = load_coco_data(max_train=100, pca_features=True)

# Print out all the keys and values from the data dictionary
for k, v in data.iteritems():
    if type(v) == np.ndarray:
        print k, type(v), v.shape, v.dtype
    else:
        print k, type(v), len(v)

idx_to_word <type 'list'> 1004
train_captions <type 'numpy.ndarray'> (100, 17) int32
val_captions <type 'numpy.ndarray'> (195954, 17) int32
train_image_idxs <type 'numpy.ndarray'> (100,) int32
val_features <type 'numpy.ndarray'> (40504, 512) float32
val_image_idxs <type 'numpy.ndarray'> (195954,) int32
train_features <type 'numpy.ndarray'> (82783, 512) float32
train_urls <type 'numpy.ndarray'> (82783,) |S63
val_urls <type 'numpy.ndarray'> (40504,) |S63
word_to_idx <type 'dict'> 1004


In [4]:
word_to_idx = data['word_to_idx']
idx_to_word = {i: w for w, i in word_to_idx.iteritems()}
small_data = sample_coco_minibatch(data, split='train', batch_size=100)  # get 100 train data
train_captions, train_features, urls = small_data
start_word = idx_to_word[train_captions[0,0]]
print "start word: ", start_word

start word:  <START>


In [5]:
import cPickle as pickle
with open("random_features.npy", 'rb') as f:
    features = pickle.load(f)
# features = np.random.rand(100, 196, 512)

In [6]:
data['train_captions'] = data['train_captions'][0:100]
data['train_features'] = features

In [12]:
model = CaptionGenerator(word_to_idx, batch_size= 100, dim_feature=[196, 512], dim_embed=128,
                                   dim_hidden=128, n_time_step=16, cell_type='lstm', dtype=tf.float32)

In [13]:
solver = CaptioningSolver(model, data, n_epochs=200, batch_size=100, update_rule='adam',
                                      learning_rate=0.004, print_every=10, save_every=10)

In [14]:
solver.train()

n_epochs:  200
n_iters_per_epoch:  1
batch size:  100
n_examples:  100
Train Loss at Epoch 0: 6886.32031
Generated Sentence: <START> a a a a a a a a a a a a a a a a
Train Loss at Epoch 10: 4866.86719
Generated Sentence: <START> a a a <END>
Train Loss at Epoch 20: 4613.75684
Generated Sentence: <START> a a a a a <END>
Train Loss at Epoch 30: 4284.09473
Generated Sentence: <START> a a a a a <UNK> <END>
Train Loss at Epoch 40: 3930.74780
Generated Sentence: <START> a <UNK> <UNK> <UNK> a <UNK> <END>
Train Loss at Epoch 50: 3555.88525
Generated Sentence: <START> a woman <UNK> on a <UNK> <END>
Train Loss at Epoch 60: 3173.27417
Generated Sentence: <START> a woman is on a <UNK> <END>
Train Loss at Epoch 70: 2780.01270
Generated Sentence: <START> a woman is on a <UNK> with a <UNK> <END>
Train Loss at Epoch 80: 2385.41699
Generated Sentence: <START> a man is on a <UNK> on a <UNK> board <END>
Train Loss at Epoch 90: 2021.03479
Generated Sentence: <START> a woman is on a <UNK> <END>
Train Loss at

In [ ]:
loss, generated_captions = model.build_model()
optimizer = tf.train.AdamOptimizer(0.004).minimize(loss)
n_epoch = 2000

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    for e in range(n_epoch):
        feed_dict =  {model.features: data['train_features'], model.captions: data['train_captions']}
        _, gen_caps, l = sess.run([optimizer, generated_captions, loss], feed_dict)
        if e % 10 == 0:
            print "loss at epoch %d: %.5f" %(e, l)
            decoded = decode_captions(gen_caps, model.idx_to_word)
            print "Generated Sentence: %s" %decoded[0]